In [14]:
import pandas as pd

In [15]:
lemma_id = 'machine_nn01'

In [16]:
df = pd.read_pickle(f'./data/extended_{lemma_id}.pickle')

In [17]:
set(df[df['provenance_type'] == "seed"].id)

{'machine_nn01-38473945',
 'machine_nn01-38474097',
 'machine_nn01-38474140',
 'machine_nn01-38474233',
 'machine_nn01-38474301',
 'machine_nn01-38474405',
 'machine_nn01-38474548',
 'machine_nn01-38474607',
 'machine_nn01-38474820',
 'machine_nn01-38474877',
 'machine_nn01-38474974',
 'machine_nn01-38475013',
 'machine_nn01-38475046',
 'machine_nn01-38475099',
 'machine_nn01-38475164',
 'machine_nn01-38475286',
 'machine_nn01-38475494',
 'machine_nn01-38475772',
 'machine_nn01-38475835',
 'machine_nn01-38475923',
 'machine_nn01-38475994',
 'machine_nn01-38476096',
 'machine_nn01-38476245',
 'machine_nn01-38476316',
 'machine_nn01-38476397',
 'machine_nn01-38476566'}

In [18]:
df.provenance_type.value_counts()

branch     7866
synonym     491
seed         26
Name: provenance_type, dtype: int64

In [40]:
def select_senses_by_provenance(sub_df: pd.DataFrame, sense_ids: set, relations: list):
    indices, senses = set(),set()
    for i, row in sub_df.iterrows():
        for oed_id, relation, prov_id in row.provenance:
            if (prov_id in sense_ids) and (relation in relations):
                senses.add(oed_id); indices.add(i)
                
    return list(indices), list(senses)

In [49]:
def obtain_senses(df, sense_ids:set, relations:list):
    seeds = df[df['provenance_type'] == "seed"].reset_index(inplace=False)
    synonyms = df[df['provenance_type'] == "synonym"].reset_index(inplace=False)
    branches = df[df['provenance_type'] == "branch"].reset_index(inplace=False)
    
    #print(seeds.shape[0],synonyms.shape[0],branches.shape[0])
    if "seed" in relations:
        seeds_selected = set(seeds[seeds.id.isin(sense_ids)].id)

    if "synonym" in relations:
        syn_sel_indices, synonyms_selected = select_senses_by_provenance(synonyms,sense_ids,relations)
    
    
    select_synonyms_semantic_class_id = synonyms[synonyms.id.isin(synonyms_selected)].semantic_class_last_id
    semantic_class_ids = set().union(*map(set,select_synonyms_semantic_class_id))
    
    branch_sel_indices, branches_selected = select_senses_by_provenance(branches,semantic_class_ids,relations)
    
    senses = set(branches.iloc[branch_sel_indices].id
               ).union(set(synonyms.iloc[syn_sel_indices].id)
                        ).union(set(seeds_selected))
    return senses
    
    
senses = obtain_senses(df,{'machine_nn01-38475835','machine_nn01-38475923',},['seed','synonym','descendant'])

In [50]:
senses

{'accumulator_nn01-34592667',
 'biocomputer_nn01-11753081',
 'commutator_nn01-8782500',
 'compressor_nn01-8608218',
 'compressor_nn01-8608248',
 'dynamo_nn01-5878672',
 'energizer_nn01-1246574970',
 'generator_nn01-135539783',
 'heatexchanger_nn01-1908252',
 'inductordisk_nn01-1192999040',
 'inductorplate_nn01-1192999070',
 'lever_nn01-39377191',
 'leverage_nn01-39378231',
 'machine_nn01-38475835',
 'machine_nn01-38475923',
 'machinepower_nn01-38476982',
 'mechanicaladvantage_nn01-37502752',
 'mover_nn01-35820685',
 'pair_nn01-32480956',
 'pairing_nn02-32485248',
 'peritrochium_nn01-30868432',
 'power_nn01-28687898',
 'powertoweightratio_nn01-28694452',
 'primemover_nn01-28348676',
 'primummobile_nn01-28369573',
 'prise_nn03-28455147',
 'purchase_nn01-27522250',
 'rocker_nn01-25157825',
 'screw_nn01-24083320',
 'screwcompressor_nn01-24085973',
 'turingmachine_nn01-17499756',
 'vice_nn02-15526247',
 'wedge_nn01-14811588',
 'wheel_nn01-14697935'}